In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"


In [23]:
from tensorflow import keras
from keras.datasets import imdb
from keras.layers import Dense,LayerNormalization,Dropout,MultiHeadAttention,GlobalAveragePooling1D,Embedding,Layer
from keras.preprocessing.sequence import pad_sequences
from keras import ops
import tensorflow as tf 

In [24]:
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=20000)

In [25]:
x_train.shape

(25000,)

In [26]:
max_len=500

x_train=pad_sequences(x_train,maxlen=max_len)
x_test=pad_sequences(x_test,maxlen=max_len)

In [27]:
x_train.shape

(25000, 500)

In [28]:
class Embedding_Position(Layer): 
    def __init__(self,vocab=20000,emb=32,max_len=500): 
        super().__init__() 
        self.token=Embedding(input_dim=vocab,output_dim=32)
        self.pos=Embedding(input_dim=max_len,output_dim=32)

    def call(self,inputs): 
        val=tf.shape(inputs)[-1]
        possitions=tf.range(start=0,limit=val)
        pos=self.pos(possitions) 
        x=self.token(inputs)
        return pos+x

In [36]:
class Transformer_Block(Layer): 
    def __init__(self,emb=32,ffn=32):
        super().__init__()
        self.att=MultiHeadAttention(num_heads=2,key_dim=emb)
        self.ff_n=keras.Sequential([Dense(ffn,activation="relu"),Dense(emb,activation="relu")])
        self.drop1=Dropout(rate=0.1) 
        self.drop2=Dropout(rate=0.1) 
        self.norm1=LayerNormalization(epsilon=1e-5)
        self.norm2=LayerNormalization(epsilon=1e-5)

    def call(self,inputs): 
        x1=self.att(inputs,inputs)
        x2=self.drop1(x1)
        x3=self.norm1(x1+x2)
        x4=self.ff_n(x3)
        x5=self.drop2(x4)
        return self.norm2(x4+x5)



In [39]:
inputs=keras.Input(shape=(500,))

embed=Embedding_Position() 
x=embed(inputs)

trans=Transformer_Block()
x=trans(x)

x=GlobalAveragePooling1D()(x)
x=Dense(256,activation="relu")(x)

outputs=Dense(2)(x)

model=keras.Model(inputs=inputs,outputs=outputs)

In [40]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
             loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
             metrics=["accuracy"])

In [41]:
model.fit(x_train,y_train,epochs=2,batch_size=32) 

Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 97s 123ms/step - accuracy: 0.5247 - loss: 0.6846
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 97s 125ms/step - accuracy: 0.9072 - loss: 0.2401


In [42]:
model.evaluate(x_test,y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 37s 47ms/step - accuracy: 0.8587 - loss: 0.3219


[0.3234214782714844, 0.8603600263595581]

In [ ]:
from keras import Te


In [63]:

import tensorflow as tf

# Sample text data
texts = ["I love machine learning", "this is the worst movie ever"]


max_tokens = 20000  
max_length = 500    
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',         
    output_sequence_length=max_length
)


vectorizer.adapt(texts)


sequences = vectorizer(texts)
print("Sequences:\n", sequences.numpy())


vocab = vectorizer.get_vocabulary()
print("Vocabulary:", vocab)

Sequences:
 [[10  7  6  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0 

In [62]:
val=sequences[0]
new=tf.reshape(val,shape=(1,500))

In [60]:
model.predict(new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step


array([[ 1.5785829, -0.6677219]], dtype=float32)